In [1]:
import pandas as pd
from confluent_kafka import Producer
import time
import json
import random
from ejercicios.pokemons import SEED, TOPIC_POKEMONS, DATA, COORDINATES

In [2]:
random.seed(SEED)

In [3]:
ds = pd.read_csv(DATA)
ds = ds.drop(labels=['#', 'Type 2', 'Total', 'Sp. Atk', 'Sp. Def', 'Generation'], axis=1)
ds = ds.rename({'Type 1': 'Type'}, axis=1)
ds.head()

,Name,Type,HP,Attack,Defense,Speed,Legendary
0,Bulbasaur,Grass,45,49,49,45,False
1,Ivysaur,Grass,60,62,63,60,False
2,Venusaur,Grass,80,82,83,80,False
3,VenusaurMega Venusaur,Grass,80,100,123,80,False
4,Charmander,Fire,39,52,43,65,False


In [4]:
pokemon = ds.sample(n=1)
as_dict = pokemon.to_dict('records')[0]

In [5]:
pokemon

,Name,Type,HP,Attack,Defense,Speed,Legendary
81,Graveler,Rock,55,95,115,35,False


In [6]:
as_dict

{'Name': 'Graveler',
 'Type': 'Rock',
 'HP': 55,
 'Attack': 95,
 'Defense': 115,
 'Speed': 35,
 'Legendary': False}

In [7]:
GAUSS_LAT_MADRID = COORDINATES['GAUSS_LAT_MADRID']
GAUSS_LON_MADRID = COORDINATES['GAUSS_LON_MADRID']
GAUSS_LAT_SEGOVIA = COORDINATES['GAUSS_LAT_SEGOVIA']
GAUSS_LON_SEGOVIA = COORDINATES['GAUSS_LON_SEGOVIA']


In [8]:
as_dict['lat'] = random.gauss(**GAUSS_LAT_MADRID)
as_dict['lon'] = random.gauss(**GAUSS_LON_MADRID)
as_dict

{'Name': 'Graveler',
 'Type': 'Rock',
 'HP': 55,
 'Attack': 95,
 'Defense': 115,
 'Speed': 35,
 'Legendary': False,
 'lat': 40.7076369506311,
 'lon': -3.0202217565200917}

In [9]:
p = Producer({'bootstrap.servers': 'localhost:9092'})

def delivery_report(err, msg):
    """ Called once for each message produced to indicate delivery result.
        Triggered by poll() or flush(). """
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered to {} [{}]'.format(msg.topic(), msg.partition()))

while True:
    pokemon = ds.sample(n=1)
    pokemon = pokemon.to_dict('records')[0]

    if random.randrange(0, 10, 1) < 5:
        pokemon['lat'] = random.gauss(**GAUSS_LAT_MADRID)
        pokemon['lon'] = random.gauss(**GAUSS_LON_MADRID)
    else:
        pokemon['lat'] = random.gauss(**GAUSS_LAT_SEGOVIA)
        pokemon['lon'] = random.gauss(**GAUSS_LON_SEGOVIA)
        

    # Trigger any available delivery report callbacks from previous produce() calls
    p.poll(0)

    # Asynchronously produce a message, the delivery report callback
    # will be triggered from poll() above, or flush() below, when the message has
    # been successfully delivered or failed permanently.
    print("Sending pokemon {Name}:{Type}:{lat}:{lon}".format(**pokemon))
    p.produce(TOPIC_POKEMONS, json.dumps(pokemon).encode('utf-8'), callback=delivery_report)
    time.sleep(abs(random.gauss(3, 1)))

# Wait for any outstanding messages to be delivered and delivery report
# callbacks to be triggered.
p.flush()


Sending pokemon Floette:Fairy:40.442550576890554:-3.1323710510753022
Message delivered to pokemons [0]
Sending pokemon Larvitar:Rock:40.54063121254469:-3.5202752094064986
Message delivered to pokemons [0]
Sending pokemon Beartic:Ice:40.795392265538915:-3.9810080469683964
Message delivered to pokemons [0]
Sending pokemon Snorunt:Ice:40.45287354468946:-3.756120160790854
Message delivered to pokemons [0]
Sending pokemon Mantyke:Water:40.972286628536885:-4.1113258299562725
Message delivered to pokemons [0]
Sending pokemon Poochyena:Dark:40.8891246765539:-4.324880865375574
Message delivered to pokemons [0]
Sending pokemon Mamoswine:Ice:40.115382050839926:-3.7173611961117414


KeyboardInterrupt: 